In [20]:
# 装饰器
import time
ts = time.sleep
get_time = time.perf_counter # 表示程序运行的时间，不包括睡眠时间
def pp(x):
    print('运行时间',int(1000*get_time()-x))
def timeit(func):
    def wrapper(*args,**kwargs):    # *args,**kwargs表示任意参数
        t = get_time()
        res = func(*args,**kwargs)  # 传参
        pp(t)
        return res
    return wrapper


In [21]:
@ timeit
def test1(xlist): 
    ts(0.5)
    return '-'.join(xlist)
@ timeit
def test2(x,y,z,w):
    ts(0.1)
    return x*y*z*w
@ timeit
def test3():
    ts(0.2)
    return 'ok'
test1(('1','3','5')),test2(2,3,4,5),test3()




运行时间 81261217
运行时间 81261325
运行时间 81261528


('1-3-5', 120, 'ok')

In [22]:
import requests
ul = 'https://www.zhihu.com'
u2 = 'www.buchunzai.com'
requests.get(ul).text[100:150]
# request.get(u2).text[100:150]
# 遇到网络不佳，对方简单反爬→简单做法就是不规则中断时间重试


'a charSet="utf-8"/><title data-rh="true">知乎 - 有问题，'

In [23]:
def try_counter(cc):
    def wrapper1(func):
        def wrapper(*args,**kwargs):
            c = 0
            while True:
                try:
                    res = func(*args,**kwargs)
                    return res
                except:
                    c += 1
                    print('crawl',c)
                    if c < cc:
                        ts(0.2)
                        continue
                    print('bad task')
                    return 0
        return wrapper
    return wrapper1


In [24]:
tt = try_counter(4)
@tt
def getweb(url):
    return requests.get(url).text[100:150]
# getweb(u1)
getweb(u2)

crawl 1
crawl 2
crawl 3
crawl 4
bad task


0

In [25]:
# case :生成标记文本
def makebold(f):
    def inner(s): 
        return '<b>'+f(s)+'</b>'
    return inner
def makeitalic(f):
    def inner(s):
        return '<i>'+f(s)+'</i>'
    return inner

@ makebold
def makeTxt(s):
    return s.upper()
makeTxt('hello, world!')




'<b>HELLO, WORLD!</b>'

函数分析工具箱: 调试阶段测性能,记录日志,算资源利用等

In [26]:
def benchmark(func):
    # 执行时间计算
    def wrapper(*args,**kwargs):    # *args,**kwargs表示任意参数
        t = get_time()
        res = func(*args,**kwargs)  # 传参
        pp(t)
        return res
    return wrapper
def counter(ff):
    # 统计执行次数
    def wrapper(*args,**kwargs):
        wrapper.count += 1
        print(f"{ff.__name__}运行了{wrapper.count}次")
        return ff(*args,**kwargs)
    wrapper.count = 0   # 万物皆对象，函数也是对象，可以添加属性
    return wrapper
    
def logging(logfile):
    # 记录运行日志
    def wrapper1(func):
        def wrapper(*args,**kwargs):
            print(f"write log to {logfile}...")
            return func(*args,**kwargs)
        return wrapper
    return wrapper1 # 两层嵌套，第一层传参，第二层传函数
    



In [27]:
def ff():
    pass
ff.cc = 0
ff.count = 999
ff.__dict__

{'cc': 0, 'count': 999}

In [28]:
import requests
@ benchmark
@ counter
@ logging('log.txt')
def getweb(url):
    try:
        return len(request.get(url).text)
    except:
        return 0
    
for url in {
    'http://httpbin.org/get',
    'www.lzu.com',
    'http://www.douban.com'
}:
    print(url)
    print('-----------------')
    print(getweb(url))

www.lzu.com
-----------------
wrapper运行了1次
write log to log.txt...
运行时间 81262992
0
http://httpbin.org/get
-----------------
wrapper运行了2次
write log to log.txt...
运行时间 81262992
0
http://www.douban.com
-----------------
wrapper运行了3次
write log to log.txt...
运行时间 81262992
0
